In [ ]:
class BankersAlgorithm:
    def __init__(self, max_resources, allocated_resources, total_resources):
        # Initialize the BankersAlgorithm class with the maximum resources required by each process,
        # currently allocated resources for each process, and total available resources in the system.
        self.max_resources = max_resources
        self.allocated_resources = allocated_resources
        self.total_resources = total_resources
        self.num_processes = len(max_resources)   # Number of processes in the system
        self.num_resources = len(total_resources) # Number of resource types in the system
        
        # Calculate the available resources by subtracting the currently allocated resources from the total resources
        self.available_resources = [total_resources[j] - sum(allocated_resources[i][j] for i in range(self.num_processes))
                                    for j in range(self.num_resources)]

    def is_safe_state(self, request, process_id):
        # Check if the requested resources exceed the maximum claim of the process
        if any(request[j] > self.max_resources[process_id][j] for j in range(self.num_resources)):
            return False

        # Check if the requested resources exceed the available resources
        if any(request[j] > self.available_resources[j] for j in range(self.num_resources)):
            return False

        # Temporarily allocate the requested resources to check for safety
        for j in range(self.num_resources):
            self.available_resources[j] -= request[j]
            self.allocated_resources[process_id][j] += request[j]

        # Perform safety check
        is_safe = self.check_safety()

        # Rollback the temporary allocation
        for j in range(self.num_resources):
            self.available_resources[j] += request[j]
            self.allocated_resources[process_id][j] -= request[j]

        return is_safe

    def check_safety(self):
        # Safety check to determine if the current system state is safe or not
        work = self.available_resources[:]  # Make a copy of available resources
        finish = [False] * self.num_processes  # Keep track of finished processes
        safe_sequence = []  # To store the sequence of processes that can finish without deadlock

        # Iterate until all processes are finished or deadlock is detected
        while True:
            found = False
            for i in range(self.num_processes):
                # Check if the current process can finish based on its needs and available resources
                if not finish[i] and all(self.max_resources[i][j] - self.allocated_resources[i][j] <= work[j]
                                         for j in range(self.num_resources)):
                    # If the process can finish, update the work and finish arrays
                    work = [work[j] + self.allocated_resources[i][j] for j in range(self.num_resources)]
                    finish[i] = True
                    safe_sequence.append(i)
                    found = True
            # If no process can finish in the current iteration, exit the loop
            if not found:
                break
        
        # If all processes are finished, the system is in a safe state
        return all(finish)

# The main function to demonstrate the Banker's algorithm
if __name__ == "__main__":
    # case 1
    max_resources = [
        [7, 5, 3],
        [3, 2, 2],
        [9, 0, 2],
        [2, 2, 2],
        [4, 3, 3]
    ]

    allocated_resources = [
        [0, 1, 0],
        [2, 0, 0],
        [3, 0, 2],
        [2, 1, 1],
        [0, 0, 2]
    ]

    total_resources = [10, 5, 7]

    banker = BankersAlgorithm(max_resources, allocated_resources, total_resources)
    request = [1, 0, 2]
    process_id = 1

    if banker.is_safe_state(request, process_id):
        print("Request is safe to be granted.")
    else:
        print("Request may lead to a deadlock and cannot be granted.")


In [ ]:
# case 2
if __name__ == "__main__":
    max_resources = [
        [7, 5, 3],
        [3, 2, 2],
        [9, 0, 2],
    ]

    allocated_resources = [
        [0, 1, 0],
        [2, 0, 0],
        [3, 0, 2],
    ]

    total_resources = [10, 5, 7]

    banker = BankersAlgorithm(max_resources, allocated_resources, total_resources)
    request = [5, 0, 0]
    process_id = 0

    if banker.is_safe_state(request, process_id):
        print("Request is safe to be granted.")
    else:
        print("Request may lead to a deadlock and cannot be granted.")





### Alternative 2
provides possible safe sequences

In [ ]:
def calculate_need(need, max_resources, allocated_resources):
    # Calculate the need matrix for each process
    for i in range(len(max_resources)):
        for j in range(len(max_resources[0])):
            # Need = Maximum resources required - Allocated resources
            need[i][j] = max_resources[i][j] - allocated_resources[i][j]

def is_safe_state(processes, available_resources, max_resources, allocated_resources):
    # Initialize the need matrix with zeros
    need = [[0 for _ in range(len(available_resources))] for _ in range(len(processes))]

    # Calculate the need matrix for each process
    calculate_need(need, max_resources, allocated_resources)

    # Create a copy of available resources to keep track of available resources during simulation
    work = available_resources[:]

    # Create a list to keep track of finished processes, initially set to 0 (not finished)
    finish = [0] * len(processes)

    # Create an empty list to store the safe sequence of processes
    safe_sequence = []

    # Loop until all processes are finished or deadlock is detected
    while True:
        found = False
        for p in range(len(processes)):
            # Check if the process is not finished and if its needs can be satisfied with available resources
            if not finish[p] and all(need[p][j] <= work[j] for j in range(len(available_resources))):
                # Update the available resources by adding the allocated resources of the process
                work = [work[j] + allocated_resources[p][j] for j in range(len(available_resources))]
                # Mark the process as finished
                finish[p] = 1
                # Add the process to the safe sequence
                safe_sequence.append(p)
                found = True

        # If no process can finish in the current iteration, exit the loop
        if not found:
            break

    # If all processes are finished, the system is in a safe state
    if all(finish):
        print("System is in safe state.")
        print("Safe sequence is:", safe_sequence)
        return True
    else:
        print("System is not in safe state.")
        return False

# Driver code
if __name__ == "__main__":
    processes = [0, 1, 2, 3, 4]
    available_resources = [3, 3, 2]

    max_resources = [
        [7, 5, 3],
        [3, 2, 2],
        [9, 0, 2],
        [2, 2, 2],
        [4, 3, 3]
    ]

    allocated_resources = [
        [0, 1, 0],
        [2, 0, 0],
        [3, 0, 2],
        [2, 1, 1],
        [0, 0, 2]
    ]

    # Check if the system is in a safe state using Banker's Algorithm
    is_safe_state(processes, available_resources, max_resources, allocated_resources)


### Alternative 3
provides all possible safe sequences

In [ ]:
def is_available(process_id, allocated, max_resources, need, available):
    # Check if all the available resources are greater than or equal to the process needs
    return all(need[process_id][i] <= available[i] for i in range(len(available)))

def safe_sequence(processes, allocated, max_resources, need, available, safe=[]):
    # Check if a safe-sequence is found and display it, or recursively find more safe sequences
    if len(safe) == len(processes):
        # If a safe-sequence is found, display it
        print("Safe sequence:", "->".join("P" + str(p + 1) for p in safe))
    else:
        for i in range(len(processes)):
            if i not in safe and is_available(i, allocated, max_resources, need, available):
                # Mark the process and update available resources
                marked = safe + [i]
                new_available = [available[j] + allocated[i][j] for j in range(len(available))]

                # Recursively find safe sequences with the updated state
                safe_sequence(processes, allocated, max_resources, need, new_available, marked)

# Driver code
if __name__ == "__main__":
    # Define the processes, allocated resources, maximum resources, and available resources
    processes = [0, 1, 2, 3]
    allocated = [[0, 1, 0], [2, 0, 0], [3, 0, 2], [2, 1, 1]]
    max_resources = [[7, 5, 3], [3, 2, 2], [9, 0, 2], [2, 2, 2]]
    resources = [10, 5, 7]

    # Calculate the need matrix and available resources
    need = [[max_resources[i][j] - allocated[i][j] for j in range(len(resources))] for i in range(len(processes))]
    available = [resources[j] - sum(allocated[i][j] for i in range(len(processes))) for j in range(len(resources))]

    # Find and print all safe sequences
    print("Safe sequences are:")
    safe_sequence(processes, allocated, max_resources, need, available)

    # Calculate the total number of possible safe sequences
    total_safe_sequences = 2 ** len(processes) - 1
    print("Total safe sequences:", total_safe_sequences)


### Alternative 4
With input

In [ ]:
def input_data():
    global n, r, max_resources, allocated_resources, available_resources

    print("********** Banker's Algorithm ************")
    n = int(input("Enter the number of Processes: "))
    r = int(input("Enter the number of resources instances: "))

    print("Enter the Max Matrix:")
    max_resources = []
    for i in range(n):
        max_resources.append(list(map(int, input().split())))

    print("Enter the Allocation Matrix:")
    allocated_resources = []
    for i in range(n):
        allocated_resources.append(list(map(int, input().split())))

    print("Enter the available Resources:")
    available_resources = list(map(int, input().split()))


def show_data():
    global n, r, max_resources, allocated_resources, available_resources

    print("\nProcess\tAllocation\tMax\tAvailable")
    for i in range(n):
        print(f"\nP{i+1}\t {allocated_resources[i]} \t{max_resources[i]}", end=" ")
        if i == 0:
            print("\t", *available_resources, end=" ")


def calculate_need():
    global n, r, max_resources, allocated_resources, need

    need = [[0] * r for _ in range(n)]

    for i in range(n):
        for j in range(r):
            need[i][j] = max_resources[i][j] - allocated_resources[i][j]


def is_available(process_id):
    global need, available_resources

    for j in range(r):
        if need[process_id][j] > available_resources[j]:
            return False

    return True


def banker_algorithm():
    global n, r, max_resources, allocated_resources, available_resources, need

    finish = [0] * n
    safe_sequence = []
    flag = True

    while flag:
        flag = False
        for i in range(n):
            if not finish[i] and is_available(i):
                for j in range(r):
                    available_resources[j] += allocated_resources[i][j]
                finish[i] = 1
                flag = True
                safe_sequence.append(i)

    if len(safe_sequence) == n:
        print("\nSafe sequence:", end=" ")
        for i in safe_sequence:
            print(f"P{i+1}", end="")
            if i != safe_sequence[-1]:
                print(" ->", end="")
        print("\n\nThe system is in a safe state.")
    else:
        print("\nProcess P", end="")
        for i in range(n):
            if not finish[i]:
                print(i + 1, end="")
                if i != n - 1:
                    print(" ->", end="")
        print(" is in deadlock.")
        print("\n\nThe system is in an unsafe state.")


if __name__ == "__main__":
    input_data()
    show_data()
    calculate_need()
    banker_algorithm()
